In [ ]:
using ITensors

In [ ]:
"""
Synthesize two MPO as M2 * M1.
When using the synthesized MPO, M1 will be applied to a MPS first, then M2.
"""
function synthesize(M1::MPO, M2::MPO)
    M21 = contract(prime(M2), M1)
    prime(M21, -1, plev=2)
end

In [ ]:
# Use two qubits
nqubit = 2

# Create site indices
sites = siteinds("Qubit", nqubit)

In [ ]:
# Hadamard gate at qubit1, identity gate at qubit 2
# (dim=2|id=666|"Qubit,Site,n=1")が状態に作用して、(dim=2|id=666|"Qubit,Site,n=1")'が出力
layer1 = MPO(sites, ["H", "I"])
@show layer1[1]
@show layer1[2]
;

In [ ]:
layer2 = MPO(sites, ["X", "I"])
@show layer2[1]
@show layer2[2]
;

In [ ]:
layer12 = synthesize(layer1, layer2)
@show layer12[1]
@show layer12[2]

In [ ]:
# Create a random MPS with bond dimension 4
M = randomMPS(sites; linkdims=4)

In [ ]:
# Apply layer1 and layer2 to M in this order
res = apply(layer2, apply(layer1, M; cutoff=1e-20); cutoff=1e-20)

# To Julia Array: (qubit1, qubit2)
res_arr = Array(reduce(*, res), sites)

In [ ]:
# Apply layer1 and layer2 to M in this order
res2 = apply(layer12, M; cutoff=1e-20) 

# To Julia Array: (qubit1, qubit2)
res2_arr = Array(reduce(*, res2), sites)

In [ ]:
# Create Z_1 X_2
# See: https://github.com/ITensor/ITensors.jl/blob/main/src/mps/mpo.jl#L69
os = Prod{Op}()
os *= Op("Z", 1)
os *= Op("X", 2)
MPO(os, sites)

In [ ]:
# Create Z_1
MPO(Op("Z", 1), sites)

In [ ]:
# Control: 1
# Target: 2
cnot = op("CNOT", sites, 1, 2)
MPO(cnot, sites)

In [ ]:
mps = MPS(sites, ["1", "0"])
@show mps[1]
@show mps[2]